In [ ]:
import os
import gc
import joblib
import numpy as np
import polars as pl
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import pad_sequences, to_categorical

from src.nn_blocks import (
    unet_se_cnn,
    features_processing, 
    GatedMixupGenerator, 
    tof_block, 
    match_time_steps, 
    time_sum, 
    squeeze_last_axis,
    expand_last_axis,
    crop_or_pad_output_shape
)

from src.functions import (
    train_model, 
    create_sequence_dataset,
    perform_padding,
    generate_gate_targets
)

from src.merge_feats_dynamic import merge_feature_sets

# =====================================================================================
# MASTER CONTROL FLAG
# =====================================================================================
TRAIN = False
TRAIN = True 

# =====================================================================================
# CONFIGURATION
# =====================================================================================
PARQUET_FILE = 'output/final_model_input_dataset.parquet'
# PARQUET_FILE = "data/extended_features_df.parquet"
# PARQUET_FILE = 'output/kaggle_0.8_feats.parquet'
PRETRAINED_DIR = Path("output/artifacts")
PRETRAINED_DIR.mkdir(parents=True, exist_ok=True) # Ensure directory exists

LR_INIT = 5e-4
WD = 3e-3
NUM_CLASSES = 18
BATCH_SIZE = 64
N_SPLITS = 4 
MAX_PAD_LEN = 128

def create_sequence_dataset_simple(df: pl.DataFrame, feature_cols: list):
    sequences = []
    labels = []
    for seq_id, group in df.group_by('sequence_id', maintain_order=True):
        sequences.append(group.select(feature_cols).to_numpy())
        labels.append(group.select('gesture_int').item(0, 0))
    return np.array(sequences, dtype=object), np.array(labels)



# =====================================================================================
# MODEL DEFINITION (Your existing function)
# =====================================================================================

from src.nn_blocks import (
    wave_block, residual_se_cnn_block, tof_block_2, attention_layer
)

def create_model(dataset, imu_dim, wd=1e-4):
    sample_batch = next(iter(dataset))
    input_shape = sample_batch[0].shape[1:]
    inp = tf.keras.layers.Input(shape=input_shape)
    imu = tf.keras.layers.Lambda(lambda t: t[:, :, :imu_dim])(inp)

    x = unet_se_cnn(imu, 3, base_filters=128, kernel_size=3)
    x = attention_layer(x) # Assuming attention_layer is defined 
    x = tf.keras.layers.Dropout(0.3)(x) 

    main_out = tf.keras.layers.Dense(18, activation="softmax", name="main_output")(x)
    return tf.keras.models.Model(inputs=inp, outputs=main_out)


# =====================================================================================
# TRAINING LOGIC
# =====================================================================================


FEATURE_DIR = Path('output')
RAW_DIR = Path('input/cmi-detect-behavior-with-sensor-data')
RANDOM_STATE = 42
from sklearn.model_selection import StratifiedKFold

if TRAIN:
    # --- Step 1: Define the feature sets to merge for this experiment ---
    files_to_merge = [
        "imu_physics_feats.parquet",
        # "imu_rolling_stats_features.parquet",
        # "imu_cross_modal_features.parquet",
    ]
    feature_paths = [FEATURE_DIR / f for f in files_to_merge]
    
    # --- Step 2: Build a LEAN Base DataFrame ---
    print("  Building a lean base DataFrame with metadata and raw IMU columns...")
    base_df = pl.read_parquet(FEATURE_DIR / "cleaned_base_train_data.parquet")
    demographics_df = pl.read_csv(RAW_DIR / "train_demographics.csv")
    base_df = base_df.join(demographics_df, on='subject', how='left')

    # --- THIS IS THE CRITICAL FIX, FOLLOWING YOUR INSTRUCTIONS ---
    # 1. Define the exact columns to keep BEFORE the merge to save memory.
    all_raw_columns = base_df.columns
    # Metadata columns that are essential for the process
    meta_cols = ['sequence_id', 'sequence_counter', 'subject', 'gesture']
    # Raw IMU columns needed by the feature engineering functions
    raw_imu_cols = [c for c in all_raw_columns if c.startswith(('acc_', 'rot_'))]
    
    # 2. Immediately narrow the DataFrame. This drops all raw ToF/Thm columns.
    base_df = base_df.select(meta_cols + raw_imu_cols)
    print(f"  Lean base DataFrame created with shape: {base_df.shape}")

    # 3. Perform Label Encoding on the lean DataFrame.
    print("  Performing label encoding...")
    le = LabelEncoder()
    gesture_encoded = le.fit_transform(base_df.get_column('gesture'))
    base_df = base_df.with_columns(pl.Series("gesture_int", gesture_encoded))  

    # --- Step 3: Run the merge function ---
    final_df = merge_feature_sets(base_df, feature_paths)
    print(f"  Final merged DataFrame created with shape: {final_df.shape}")

    # --- Step 4: Define FINAL Feature Columns for the Model ---
    all_final_columns = final_df.columns
    # Define all columns that are NOT features for the model
    final_meta_cols = {'gesture', 'gesture_int', 'sequence_type', 'behavior', 'orientation',
                       'row_id', 'subject', 'phase', 'sequence_id', 'sequence_counter'}
    demographic_cols = {'adult_child', 'age', 'sex', 'handedness', 'height_cm', 'shoulder_to_wrist_cm', 'elbow_to_wrist_cm'}
    
    # This is the final list of columns to be scaled and fed to the model
    imu_cols = [c for c in all_final_columns if c not in final_meta_cols and c not in demographic_cols]
    imu_dim = len(imu_cols)
    print(f"  Training with {imu_dim} final IMU features.")    

    # --- Step 5: Prepare for Cross-Validation ---
    cv_info = final_df.group_by("sequence_id").agg(pl.first("gesture_int")).sort("sequence_id")
    all_sequence_ids = cv_info.get_column("sequence_id").to_numpy()
    y_for_split = cv_info.get_column("gesture_int").to_numpy()
    
    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    fold_accuracies = []
    all_preds = []
    all_labels = []

    for fold_idx, (train_indices, val_indices) in enumerate(kf.split(all_sequence_ids, y_for_split)):
        print(f"\n=== Fold {fold_idx + 1}/{N_SPLITS} ===")
        train_ids = all_sequence_ids[train_indices]
        val_ids = all_sequence_ids[val_indices]

        # Filter the merged DataFrame for the current fold
        train_df = final_df.filter(pl.col('sequence_id').is_in(train_ids))
        val_df = final_df.filter(pl.col('sequence_id').is_in(val_ids))
        print("Fold data loaded.")

        # Label encoding is already done, but we need the encoder for the final report
        le = LabelEncoder().fit(train_df['gesture'])
        
        # --- StandardScaler Logic ---
        scaler = StandardScaler()
        train_features_scaled = scaler.fit_transform(train_df[imu_cols])
        val_features_scaled = scaler.transform(val_df[imu_cols])
        X_train_scaled_features = pl.DataFrame(train_features_scaled, schema=imu_cols)
        X_val_scaled_features = pl.DataFrame(val_features_scaled, schema=imu_cols)

        meta_cols_to_keep = ['sequence_id', 'gesture_int']
        train_df_final = train_df.select(meta_cols_to_keep).with_columns(X_train_scaled_features)
        val_df_final = val_df.select(meta_cols_to_keep).with_columns(X_val_scaled_features)

        del train_df, val_df, X_train_scaled_features, X_val_scaled_features
        gc.collect()

        # Create sequences (no gate target needed)
        X_train, y_train = create_sequence_dataset_simple(train_df_final, imu_cols)
        X_val, y_val = create_sequence_dataset_simple(val_df_final, imu_cols)

        del train_df_final, val_df_final
        gc.collect()

        X_train_padded = pad_sequences(X_train, maxlen=MAX_PAD_LEN, padding='post', truncating='post', dtype='float32')
        X_val_padded = pad_sequences(X_val, maxlen=MAX_PAD_LEN, padding='post', truncating='post', dtype='float32')
        
        y_train_cat = to_categorical(y_train, num_classes=NUM_CLASSES)
        y_val_cat = to_categorical(y_val, num_classes=NUM_CLASSES)

        # Create simple TF Datasets (no generator needed unless you want mixup)
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train_padded, y_train_cat)).shuffle(len(X_train_padded)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        val_dataset = tf.data.Dataset.from_tensor_slices((X_val_padded, y_val_cat)).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

        del X_train, y_train, X_val, y_val, X_train_padded, X_val_padded
        gc.collect()
        
        # Use the new IMU-only model
        model = create_model(train_dataset, imu_dim)
        
        # Adapt the train_model call for a single output
        optimizer = tf.keras.optimizers.AdamW(learning_rate=LR_INIT, weight_decay=WD)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max')
        model.fit(train_dataset, validation_data=val_dataset, epochs=150, callbacks=[early_stopping])
        
        # --- EVALUATION ---
        # The model now returns a single array, not a dictionary
        y_pred_proba = model.predict(val_dataset)
        y_pred_fold = np.argmax(y_pred_proba, axis=1)
        y_true_fold = np.argmax(y_val_cat, axis=1)

        # --- EVALUATION ---
        val_preds = model.predict(val_dataset)
        main_output_preds = val_preds['main_output']
        y_pred_fold = np.argmax(main_output_preds, axis=1)
        y_true_fold = np.argmax(y_val_cat, axis=1)
        fold_acc = accuracy_score(y_true_fold, y_pred_fold)
        fold_accuracies.append(fold_acc)
        print(f"Fold {fold_idx + 1} Accuracy: {fold_acc:.4f}")
        all_preds.append(y_pred_fold)
        all_labels.append(y_true_fold)

        del train_dataset, model, val_dataset
        gc.collect()

    # --- FINAL OOF REPORT ---
    print("\n=== Cross-validation Summary ===")
    print(f"Per-fold Accuracies: {fold_accuracies}")
    print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")
    y_all_pred = np.concatenate(all_preds)
    y_all_true = np.concatenate(all_labels)
    print("\n=== Overall Classification Report ===")
    print(classification_report(y_all_true, y_all_pred, target_names=le.classes_, digits=4))

  Building a lean base DataFrame with metadata and raw IMU columns...
  Lean base DataFrame created with shape: (574945, 11)
  Performing label encoding...
 Starting merge process...
  Loading and joining features from: imu_physics_feats.parquet
  Merge complete.
  Final merged DataFrame created with shape: (574945, 32)
  Training with 27 final IMU features.

=== Fold 1/4 ===
Fold data loaded.


I0000 00:00:1755000657.235487  900300 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4714 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/150


I0000 00:00:1755000672.761059  900568 service.cc:152] XLA service 0x7e4cd0005270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755000672.761107  900568 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1060, Compute Capability 6.1
2025-08-12 13:11:13.255206: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755000675.392422  900568 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-12 13:11:19.758378: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-08-12 13:11:19.884405: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out

 3/96 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.0556 - loss: 3.3639  

I0000 00:00:1755000696.169763  900568 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


96/96 ━━━━━━━━━━━━━━━━━━━━ 70s 347ms/step - accuracy: 0.1267 - loss: 3.0998 - val_accuracy: 0.2134 - val_loss: 2.7073
Epoch 2/150
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.2389 - loss: 2.4784 - val_accuracy: 0.2969 - val_loss: 2.2885
Epoch 3/150
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.3003 - loss: 2.2262 - val_accuracy: 0.3430 - val_loss: 2.0497
Epoch 4/150
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.3497 - loss: 2.0368 - val_accuracy: 0.3832 - val_loss: 1.9248
Epoch 5/150
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.3911 - loss: 1.8910 - val_accuracy: 0.4338 - val_loss: 1.7291
Epoch 6/150
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.4244 - loss: 1.7565 - val_accuracy: 0.4490 - val_loss: 1.6352
Epoch 7/150
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.4593 - loss: 1.6704 - val_accuracy: 0.4907 - val_loss: 1.5647
Epoch 8/150
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.4769 - loss: 1.6254 - val_accuracy: 0.4799 - val

In [ ]:
for e in train_dataset:
    x = e[0]
    y = e[1]
    break

NameError: name 'train_dataset' is not defined

In [ ]:
imu_cols

['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']

In [ ]:
base_df.head()

gesture,gesture_int,sequence_type,behavior,orientation,row_id,subject,phase,sequence_id,sequence_counter,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
str,i64,str,str,str,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64
"""Cheek - pinch skin""",1,"""Target""","""Relaxes and moves hand to targ…","""Seated Lean Non Dom - FACE DOW…","""SEQ_000007_000000""","""SUBJ_059520""","""Transition""","""SEQ_000007""",0,6.683594,6.214844,3.355469,0.134399,-0.355164,-0.447327,-0.809753,0,12,1,1,163.0,52,24.0
"""Cheek - pinch skin""",1,"""Target""","""Relaxes and moves hand to targ…","""Seated Lean Non Dom - FACE DOW…","""SEQ_000007_000001""","""SUBJ_059520""","""Transition""","""SEQ_000007""",1,6.949219,6.214844,3.125,0.143494,-0.340271,-0.42865,-0.824524,0,12,1,1,163.0,52,24.0
"""Cheek - pinch skin""",1,"""Target""","""Relaxes and moves hand to targ…","""Seated Lean Non Dom - FACE DOW…","""SEQ_000007_000002""","""SUBJ_059520""","""Transition""","""SEQ_000007""",2,5.722656,5.410156,5.421875,0.219055,-0.274231,-0.356934,-0.865662,0,12,1,1,163.0,52,24.0
"""Cheek - pinch skin""",1,"""Target""","""Relaxes and moves hand to targ…","""Seated Lean Non Dom - FACE DOW…","""SEQ_000007_000003""","""SUBJ_059520""","""Transition""","""SEQ_000007""",3,6.6015625,3.53125,6.457031,0.297546,-0.26416,-0.238159,-0.885986,0,12,1,1,163.0,52,24.0
"""Cheek - pinch skin""",1,"""Target""","""Relaxes and moves hand to targ…","""Seated Lean Non Dom - FACE DOW…","""SEQ_000007_000004""","""SUBJ_059520""","""Transition""","""SEQ_000007""",4,5.566406,0.277344,9.6328125,0.333557,-0.218628,-0.063538,-0.914856,0,12,1,1,163.0,52,24.0


In [ ]:
input_shape = x[0].shape
inp = tf.keras.layers.Input(shape=input_shape)
imu = tf.keras.layers.Lambda(lambda t: t[:, :, :imu_dim])(inp)
tof = tf.keras.layers.Lambda(lambda t: t[:, :, imu_dim:])(inp)
print(input_shape)

(128, 38)


In [ ]:
wd = 0
# TOF/Thermal lighter branch
x2 = tf.keras.layers.Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(wd))(tof)
x2 = tf.keras.layers.BatchNormalization()(x2); x2 = tf.keras.layers.Activation('relu')(x2)
x2 = tf.keras.layers.MaxPooling1D(2)(x2); x2 = tf.keras.layers.Dropout(0.2)(x2)
x2 = tf.keras.layers.Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(wd))(x2)
x2 = tf.keras.layers.BatchNormalization()(x2); x2 = tf.keras.layers.Activation('relu')(x2)
x2 = tf.keras.layers.MaxPooling1D(2)(x2); x2 = tf.keras.layers.Dropout(0.2)(x2)

x2.shape

(None, 32, 128)

In [ ]:
def create_model(dataset, imu_dim, wd=1e-4):
    sample_batch = next(iter(dataset))
    input_shape = sample_batch[0].shape[1:]
    inp = tf.keras.layers.Input(shape=input_shape)
    imu = tf.keras.layers.Lambda(lambda t: t[:, :, :imu_dim])(inp)
    tof = tf.keras.layers.Lambda(lambda t: t[:, :, imu_dim:])(inp)

    # IMU deep branch
    x1 = residual_se_cnn_block(imu, 64, 3, drop=0.1, wd=wd)
    x1 = residual_se_cnn_block(x1, 128, 5, drop=0.1, wd=wd)

    # TOF/Thermal lighter branch
    x2 = tf.keras.layers.Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=tf.keras.layers.l2(wd))(tof)
    x2 = tf.keras.layers.BatchNormalization()(x2); x2 = tf.keras.layers.Activation('relu')(x2)
    x2 = tf.keras.layers.MaxPooling1D(2)(x2); x2 = tf.keras.layers.Dropout(0.2)(x2)
    x2 = tf.keras.layers.Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=tf.keras.layers.l2(wd))(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2); x2 = tf.keras.layers.Activation('relu')(x2)
    x2 = tf.keras.layers.MaxPooling1D(2)(x2); x2 = tf.keras.layers.Dropout(0.2)(x2)

    merged = tf.keras.layers.Concatenate()([x1, x2])

    xa = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, kernel_regularizer=tf.keras.layers.l2(wd)))(merged)
    xb = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, kernel_regularizer=tf.keras.layers.l2(wd)))(merged)
    xc = tf.keras.layers.GaussianNoise(0.09)(merged)
    xc = tf.keras.layers.Dense(16, activation='elu')(xc)
    
    x = tf.keras.layers.Concatenate()([xa, xb, xc])
    x = tf.keras.layers.Dropout(0.4)(x)
    x = attention_layer(x)

    for units, drop in [(256, 0.5), (128, 0.3)]:
        x = tf.keras.layers.Dense(units, use_bias=False, kernel_regularizer=tf.keras.layers.l2(wd))(x)
        x = tf.keras.layers.BatchNormalization()(x); x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.Dropout(drop)(x)

    main_out = tf.keras.layers.tf.keras.layers.Dense(18, activation="softmax", name="main_output")(x)
    gate_out = tf.keras.layers.tf.keras.layers.Dense(1, activation="sigmoid", name="tof_gate")(x) # Renamed layer
    
    return tf.keras.models.Model(inputs=inp, outputs={"main_output": main_out, "tof_gate": gate_out})



In [ ]:

# # =====================================================================================
# # --- INFERENCE & LOCAL DEBUGGING SCRIPT ---
# # =====================================================================================
# import pandas as pd
# import polars as pl
# import numpy as np
# import joblib
# import traceback
# from tensorflow.keras.models import load_model
# from tensorflow.keras.utils import pad_sequences

# import os
# import gc
# import joblib
# import numpy as np
# import polars as pl
# import tensorflow as tf
# from pathlib import Path
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.metrics import classification_report, accuracy_score
# from tensorflow.keras.models import load_model
# from tensorflow.keras.utils import pad_sequences, to_categorical
# from tensorflow import argmax, minimum, shape

# # --- Your existing function imports ---
# from src.nn_blocks import (
#     unet_se_cnn,
#     features_processing, 
#     GatedMixupGenerator, 
#     tof_block, 
#     match_time_steps, 
#     time_sum, 
#     squeeze_last_axis,
#     expand_last_axis,
#     crop_or_pad_output_shape
# )

# from src.functions import (
#     train_model, 
#     create_sequence_dataset,
#     perform_padding,
#     generate_gate_targets
# )
# from src.constants import DATA_PATH
# from src.tof_feats import remove_gravity_from_acc, calculate_angular_velocity_from_quat, calculate_angular_distance

# def crop_or_pad(inputs):
#     x, skip = inputs
#     x_len = shape(x)[1]
#     skip_len = shape(skip)[1]
#     min_len = minimum(x_len, skip_len)
#     return x[:, :min_len, :], skip[:, :min_len, :]

# # =====================================================================================
# # MASTER CONTROL FLAG
# # =====================================================================================
# TRAIN = True 
# TRAIN = False

# # =====================================================================================
# # CONFIGURATION
# # =====================================================================================
# PARQUET_FILE = 'output/final_processed_train_data.parquet'
# PRETRAINED_DIR = Path("output/artifacts")
# PRETRAINED_DIR.mkdir(parents=True, exist_ok=True) # Ensure directory exists

# LR_INIT = 5e-4
# WD = 3e-3
# NUM_CLASSES = 18
# BATCH_SIZE = 64
# N_SPLITS = 4 
# MAX_PAD_LEN = 128

# # --- 2. Define TTA Parameters and Predict Function ---
# TTA_STEPS = 10
# TTA_NOISE_STDDEV = 0.01

# # =====================================================================================
# # MODEL DEFINITION (Your existing function)
# # =====================================================================================
# def create_model(dataset, imu_dim, wd=1e-4):
#     sample_batch = next(iter(dataset))
#     input_shape = sample_batch[0].shape[1:]
#     inp = tf.keras.layers.Input(shape=input_shape)
#     imu = tf.keras.layers.Lambda(lambda t: t[:, :, :imu_dim])(inp)
#     tof = tf.keras.layers.Lambda(lambda t: t[:, :, imu_dim:])(inp)

#     x1 = unet_se_cnn(imu, 3, base_filters=64, kernel_size=3)
#     x2 = tof_block(tof, wd)

#     x = features_processing(x1, x2)
#     x = tf.keras.layers.Dropout(0.3)(x) 
#     main_out = tf.keras.layers.Dense(18, activation="softmax", name="main_output")(x)
#     gate_out = tf.keras.layers.Dense(1, activation="sigmoid", name="tof_gate")(x) # Renamed layer
    
#     return tf.keras.models.Model(inputs=inp, outputs={"main_output": main_out, "tof_gate": gate_out})

# # --- 1. Load All Inference Artifacts ---
# print("▶ LOCAL DEBUG MODE – loading artefacts from", PRETRAINED_DIR)
# try:
#     final_feature_cols = np.load(PRETRAINED_DIR / "feature_cols.npy", allow_pickle=True).tolist()
#     pad_len = int(np.load(PRETRAINED_DIR / "sequence_maxlen.npy"))
#     scaler = joblib.load(PRETRAINED_DIR / "scaler.pkl")
#     gesture_classes = np.load(PRETRAINED_DIR / "gesture_classes.npy", allow_pickle=True)

#     models = []
#     print(f"  Loading {N_SPLITS} models for ensemble inference...")
#     for fold in range(N_SPLITS):
#         model_path = PRETRAINED_DIR / f"gesture_model_fold_{fold}.h5"
#         model = load_model(model_path, compile=False, custom_objects={
#             'unet_se_cnn': unet_se_cnn,
#             'tof_block': tof_block,
#             'features_processing': features_processing,
#             'match_time_steps': match_time_steps,
#             'crop_or_pad': crop_or_pad,
#             'squeeze_last_axis': squeeze_last_axis,
#             'expand_last_axis': expand_last_axis,
#             'time_sum': time_sum,
#             'crop_or_pad_output_shape': crop_or_pad_output_shape
#         })
#         models.append(model)
#     print("  Models, scaler, and metadata loaded.")
# except Exception as e:
#     print(f"ERROR loading artifacts: {e}")
#     # Stop execution if artifacts can't be loaded
#     exit()

# # --- 2. Define the Predict Function (Using the most robust version) ---
# def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
#     # ... (All your feature engineering code is correct and can remain the same) ...
#     df_seq = sequence.to_pandas()
#     # ... (Sanitization, feature creation, scaling, padding) ...
#     sensor_cols = [c for c in df_seq.columns if c.startswith(('acc_', 'rot_', 'thm_', 'tof_'))]
#     for col in sensor_cols:
#         if df_seq[col].dtype == 'object':
#             df_seq[col] = pd.to_numeric(df_seq[col], errors='coerce')
#     new_features = {}
#     linear_accel = remove_gravity_from_acc(df_seq, df_seq)
#     new_features['linear_acc_x'] = linear_accel[:, 0]
#     new_features['linear_acc_y'] = linear_accel[:, 1]
#     new_features['linear_acc_z'] = linear_accel[:, 2]
#     linear_acc_mag = np.sqrt(np.square(linear_accel).sum(axis=1))
#     new_features['linear_acc_mag'] = linear_acc_mag
#     new_features['linear_acc_mag_jerk'] = pd.Series(linear_acc_mag).diff().fillna(0).values
#     angular_vel = calculate_angular_velocity_from_quat(df_seq)
#     new_features['angular_vel_x'] = angular_vel[:, 0]
#     new_features['angular_vel_y'] = angular_vel[:, 1]
#     new_features['angular_vel_z'] = angular_vel[:, 2]
#     new_features['angular_distance'] = calculate_angular_distance(df_seq)
#     for i in range(1, 6):
#         pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]
#         tof_data = df_seq[pixel_cols].replace(-1, np.nan)
#         new_features[f'tof_{i}_mean'] = tof_data.mean(axis=1)
#         new_features[f'tof_{i}_std'] = tof_data.std(axis=1)
#         new_features[f'tof_{i}_min'] = tof_data.min(axis=1)
#         new_features[f'tof_{i}_max'] = tof_data.max(axis=1)
#     df_seq = df_seq.assign(**new_features)
#     mat_unscaled_df = df_seq[final_feature_cols].ffill().bfill().fillna(0)
#     mat_scaled = scaler.transform(mat_unscaled_df)
#     pad_input = pad_sequences([mat_scaled], maxlen=pad_len, padding='post', truncating='post', dtype='float32')

#     # --- TTA Loop ---
#     all_tta_predictions = []
#     for i in range(TTA_STEPS):
#         noisy_input = pad_input
#         if i > 0:
#             noise = tf.random.normal(shape=tf.shape(pad_input), mean=0.0, stddev=TTA_NOISE_STDDEV)
#             noisy_input = pad_input + noise

#         # Ensemble predictions from all fold models
#         all_fold_predictions = []
#         for model in models:
            
#             # =========================================================================
#             # --- THE FINAL FIX IS HERE ---
#             # =========================================================================
#             # model.predict returns a dictionary, access the 'main_output' key
#             predictions_dict = model.predict(noisy_input, verbose=0)
#             main_preds = predictions_dict['main_output']
            
#             all_fold_predictions.append(main_preds)
        
#         avg_fold_prediction = np.mean(all_fold_predictions, axis=0)
#         all_tta_predictions.append(avg_fold_prediction)

#     # --- Final Averaging and Prediction (Unchanged) ---
#     final_avg_prediction = np.mean(all_tta_predictions, axis=0)
#     idx = int(final_avg_prediction.argmax())
    
#     return str(gesture_classes[idx])

# # =====================================================================================
# # --- LOCAL TEST HARNESS ---
# # =====================================================================================
# print("\n--- Starting Local Test ---")

# # Load the actual test data
# TEST_CSV_PATH = 'input/cmi-detect-behavior-with-sensor-data/test.csv'
# TEST_DEM_PATH = 'input/cmi-detect-behavior-with-sensor-data/test_demographics.csv'

# try:
#     test_df = pl.read_csv(TEST_CSV_PATH)
#     test_dem_df = pl.read_csv(TEST_DEM_PATH)
    
#     # Pick the first sequence from the test set
#     target_sequence_id = test_df.get_column("sequence_id").unique()[0]
#     print(f"Testing with sequence_id: {target_sequence_id}")
    
#     # Isolate the data for that single sequence
#     sample_sequence_pl = test_df.filter(pl.col("sequence_id") == target_sequence_id)
    
#     # Find the corresponding subject and their demographics
#     subject_id = sample_sequence_pl.get_column("subject")[0]
#     sample_demographics_pl = test_dem_df.filter(pl.col("subject") == subject_id)
    
#     # --- Call the predict function directly and catch the REAL error ---
#     print("\nCalling predict function directly...")
#     predicted_gesture = predict(sample_sequence_pl, sample_demographics_pl)
    
#     print("\n✅ SUCCESS! The function ran without errors on a sample.")
#     print(f"Predicted Gesture: {predicted_gesture}")

# except Exception as e:
#     print("\n❌ ERROR! The function failed. Here is the full Python traceback:")
#     traceback.print_exc()